# 실습: 3-에이전트 논문 추천 시스템

## 목표
이 실습에서는 3개의 에이전트가 협력하는 논문 추천 시스템을 만듭니다:
1. **검색 에이전트**: 사용자 질문과 관련된 논문 5개 검색
2. **요약 에이전트**: 검색된 논문들을 분석하고 요약
3. **추천 에이전트**: 분석 결과를 바탕으로 최적의 논문 3개 추천

## TODO 개수: 3개
- TODO 1: 3개의 도구 만들기
- TODO 2: 3개의 에이전트 함수 만들기
- TODO 3: LangGraph 워크플로우 구성

**예상 소요 시간: 약 1시간**

## 준비 단계: 라이브러리 설치 (이미 완료)

In [1]:
!pip install -q qdrant-client transformers torch accelerate sentence-transformers pandas langchain langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.6/328.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 21.6 MB/s eta 0:00:00


## 준비 단계: 라이브러리 임포트 (이미 완료)

In [2]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List, Any, Optional
import operator
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 임포트 완료")

라이브러리 임포트 완료


## 준비 단계: 데이터 로드 및 전처리 (이미 완료)

In [3]:
df = pd.read_csv('arxiv_ai.csv')
df = df[['title', 'summary', 'published', 'authors']].dropna()
df = df.sample(500, random_state=42).reset_index(drop=True)
df['published_year'] = pd.to_datetime(df['published']).dt.year
df['text'] = df['title'] + ' ' + df['summary']

print(f"전처리 완료: {len(df)}개 논문")

전처리 완료: 500개 논문


## 준비 단계: 임베딩 모델 로드 (이미 완료)

In [4]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print(f"임베딩 모델 로드 완료")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

임베딩 모델 로드 완료


## 준비 단계: Qdrant 설정 및 데이터 저장 (이미 완료)

In [5]:
client = QdrantClient(":memory:")
collection_name = "arxiv_papers"

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

points = []
for idx, row in df.iterrows():
    vector = embedding_model.encode(row['text']).tolist()
    point = PointStruct(
        id=idx,
        vector=vector,
        payload={
            "title": row['title'],
            "summary": row['summary'],
            "year": int(row['published_year']),
            "authors": row['authors']
        }
    )
    points.append(point)
    if (idx + 1) % 100 == 0:
        print(f"{idx + 1}개 논문 처리 완료")

client.upsert(collection_name=collection_name, points=points)
print(f"\n{len(points)}개 논문이 벡터 DB에 저장됨")

100개 논문 처리 완료
200개 논문 처리 완료
300개 논문 처리 완료
400개 논문 처리 완료
500개 논문 처리 완료

500개 논문이 벡터 DB에 저장됨


## 준비 단계: Granite 모델 로드 (이미 완료)

In [6]:
model_path = "ibm-granite/granite-4.0-micro"
tokenizer = AutoTokenizer.from_pretrained(model_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
llm_model = AutoModelForCausalLM.from_pretrained(model_path, device_map=device if device == "cuda" else None)
llm_model.eval()

def generate_response(prompt: str, max_tokens: int = 300) -> str:
    chat = [{"role": "user", "content": prompt}]
    chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    input_tokens = tokenizer(chat_text, return_tensors="pt").to(device)
    output = llm_model.generate(**input_tokens, max_new_tokens=max_tokens)
    response = tokenizer.decode(output[0], skip_special_tokens=False)
    assistant_response = response.split("<|start_of_role|>assistant<|end_of_role|>")[-1]
    assistant_response = assistant_response.split("<|end_of_text|>")[0].strip()
    return assistant_response

print(f"Granite 모델 로드 완료 ({device})")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Granite 모델 로드 완료 (cuda)


---
# 실습 시작
---

## TODO 1: 3개의 도구(Tools) 만들기

에이전트가 사용할 3개의 도구를 만드세요.

In [11]:
# TODO 1-1 (Korean): search_papers_tool을 구현하세요
# TODO 1-1 (English): Implement search_papers_tool
# HINT: @tool 데코레이터를 사용하세요
# HINT: def search_papers_tool(query: str, top_k: int = 5) -> str:
# HINT: client.query_points()로 검색하고 .points로 결과를 받으세요
# HINT: 각 논문의 제목, 연도, 유사도, 요약(150자)을 포함하여 반환하세요

@tool
def search_papers_tool(query: str, top_k: int = 5) -> str:
    """벡터 데이터베이스에서 쿼리와 관련된 논문을 검색합니다."""
    query_vector = embedding_model.encode(query).tolist()

    results = client.query_points(
        collection_name=collection_name,
        query=query_vector,
        limit=top_k
    ).points

    output = f"검색 결과 ({len(results)}개 논문):\n\n"
    for i, result in enumerate(results, 1):
        output += f"[{i}] 제목: {result.payload['title']}\n"
        output += f"    연도: {result.payload['year']}\n"
        output += f"    요약: {result.payload['summary'][:150]}...\n"
        output += f"    유사도: {result.score:.3f}\n\n"

    return output



# TODO 1-2 (Korean): summarize_papers_tool을 구현하세요
# TODO 1-2 (English): Implement summarize_papers_tool
# HINT: def summarize_papers_tool(papers_info: str) -> str:
# HINT: generate_response()를 사용하여 논문들을 분석하세요
# HINT: 프롬프트에 "핵심 기술", "적용 가능성", "연관성"을 분석하도록 요청하세요
@tool
def summarize_papers_tool(topic: str) -> str:
    """특정 주제에 대한 논문들을 검색하고 심층 분석하여 요약합니다."""
    query_vector = embedding_model.encode(topic).tolist()

    results = client.query_points(
        collection_name=collection_name,
        query=query_vector,
        limit=3
    ).points

    context = f"논문 분석 요청: {topic}\n\n"
    context += "검색된 논문들:\n"
    for i, result in enumerate(results, 1):
        context += f"\n[논문 {i}]\n"
        context += f"제목: {result.payload['title']}\n"
        context += f"출판연도: {result.payload['year']}\n"
        context += f"초록: {result.payload['summary'][:400]}\n"

    prompt = f"""{context}

위 논문들의 초록을 바탕으로 다음을 분석해주세요:

1. 핵심 연구 내용: 각 논문이 다루는 주요 기술과 방법론
2. 실용성 평가: 실제 프로젝트 적용 가능성
3. 추천 순위: 가장 유용한 논문부터 순서대로
4. 다음 단계: 이 논문들을 읽은 후 추가로 알아보면 좋을 주제

간결하고 실용적으로 답변해주세요."""

    summary = generate_response(prompt, max_tokens=400)

    return summary

tools = [search_papers_tool, summarize_papers_tool]
print("도구 정의 완료")


# TODO 1-3 (Korean): recommend_papers_tool을 구현하세요
# TODO 1-3 (English): Implement recommend_papers_tool
# HINT: def recommend_papers_tool(summary: str, papers_info: str) -> str:
# HINT: summary와 papers_info를 모두 받아서 최고의 논문 3개를 추천하세요
# HINT: 추천 이유, 활용 방법, 읽는 순서를 포함하도록 프롬프트를 작성하세요
@tool
def recommend_papers_tool(summary: str, papers_info: str) -> str:
    """
    논문 요약과 검색 정보를 바탕으로 최고의 논문 3개를 추천합니다.
    각 추천 논문에 대해 추천 이유, 활용 방법, 읽는 순서를 안내합니다.
    """
    prompt = f"""
다음은 논문 요약 및 검색된 논문 목록입니다.

[논문 분석 요약]
{summary}

[논문 세부 정보]
{papers_info}

위 논문들을 바탕으로,
1. 프로젝트/공부에 가장 도움이 될 논문 3개를 추천해 주세요.
2. 각 추천 논문의 추천 이유와 실제 활용할 수 있는 방법을 간단히 써 주세요.
3. 읽는 순서를 제안해 주세요 (예: 1→2→3).
4. 추가적으로 참고하면 좋은 논문/정보가 있으면 짧게 덧붙여 주세요.

간결하고 실용적으로 답변해 주세요.
"""

    response = generate_response(prompt, max_tokens=400)

    return response


tools = [search_papers_tool, summarize_papers_tool, recommend_papers_tool]
print(" TODO 1 완료: 3개 도구 생성")

도구 정의 완료
 TODO 1 완료: 3개 도구 생성


## TODO 2: 3개의 에이전트 함수 만들기

각 에이전트는 이전 에이전트의 결과를 받아서 처리합니다.

In [12]:
class AgentState(TypedDict):
    query: str
    search_results: str
    summary: str
    recommendations: str
    next_step: str
    messages: list

# TODO 2-1 (Korean): search_agent 함수를 구현하세요
# TODO 2-1 (English): Implement search_agent function
# HINT: def search_agent(state: AgentState) -> AgentState:
# HINT: state["query"]를 사용하여 search_papers_tool을 호출하세요
# HINT: 결과를 state["search_results"]에 저장하세요
# HINT: state["next_step"] = "summarize"로 설정하세요
def search_agent(state: AgentState) -> AgentState:
    """논문 검색 에이전트"""
    query = state["query"]
    print(f"\n[검색 에이전트] '{query}' 검색 중...")

    result = search_papers_tool.invoke({"query": query, "top_k": 3})

    state["search_results"] = result
    state["next_step"] = "summarize"
    state["messages"].append(f"검색 완료: {query}")

    return state



# TODO 2-2 (Korean): summarize_agent 함수를 구현하세요
# TODO 2-2 (English): Implement summarize_agent function
# HINT: def summarize_agent(state: AgentState) -> AgentState:
# HINT: state["search_results"]를 사용하여 summarize_papers_tool을 호출하세요
# HINT: 결과를 state["summary"]에 저장하세요
# HINT: state["next_step"] = "recommend"로 설정하세요
def summarize_agent(state: AgentState) -> AgentState:
    """논문 요약 에이전트"""
    query = state["query"]
    print(f"\n[요약 에이전트] '{query}' 요약 중...")

    summary = summarize_papers_tool.invoke({"topic": query})

    state["summary"] = summary
    state["next_step"] = "recommend"
    state["messages"].append("요약 완료")

    return state


# TODO 2-3 (Korean): recommend_agent 함수를 구현하세요
# TODO 2-3 (English): Implement recommend_agent function
# HINT: def recommend_agent(state: AgentState) -> AgentState:
# HINT: state["summary"]와 state["search_results"]를 모두 recommend_papers_tool에 전달하세요
# HINT: 결과를 state["recommendations"]에 저장하세요
# HINT: state["next_step"] = "end"로 설정하세요
def recommend_agent(state: AgentState) -> AgentState:
    """논문 추천 에이전트"""
    summary = state["summary"]
    search_results = state["search_results"]
    print(f"\n[추천 에이전트] 논문 추천 진행 중...")

    recommendations = recommend_papers_tool.invoke({
        "summary": summary,
        "papers_info": search_results
    })

    state["recommendations"] = recommendations
    state["next_step"] = "end"
    state["messages"].append("추천 완료")

    return state


def router(state: AgentState) -> str:
    return state["next_step"]

print("TODO 2 완료: 3개 에이전트 생성")

TODO 2 완료: 3개 에이전트 생성


## TODO 3: LangGraph 워크플로우 구성

3개 에이전트를 연결하는 워크플로우를 만드세요.

In [13]:
# TODO 3 (Korean): LangGraph 워크플로우를 구성하세요
# TODO 3 (English): Configure LangGraph workflow
# HINT: workflow = StateGraph(AgentState)
# HINT: workflow.add_node()로 3개 에이전트 추가 ("search", "summarize", "recommend")
# HINT: workflow.set_entry_point("search")로 시작점 설정
# HINT: workflow.add_conditional_edges()로 조건부 엣지 추가
#       - "search" → router → {"summarize": "summarize", "end": END}
#       - "summarize" → router → {"recommend": "recommend", "end": END}
# HINT: workflow.add_edge("recommend", END)로 마지막 엣지 추가
# HINT: app = workflow.compile()

workflow = StateGraph(AgentState)

workflow.add_node("search", search_agent)
workflow.add_node("summarize", summarize_agent)
workflow.add_node("recommend", recommend_agent)

workflow.set_entry_point("search")

workflow.add_conditional_edges(
    "search",
    router,
    {
        "summarize": "summarize",
        "end": END
    }
)

workflow.add_conditional_edges(
    "summarize",
    router,
    {
        "recommend": "recommend",
        "end": END
    }
)

workflow.add_edge("recommend", END)

app = workflow.compile()

print("멀티 에이전트 워크플로우 구성 완료")
print("TODO 3 완료: 워크플로우 구성 완료")

멀티 에이전트 워크플로우 구성 완료
TODO 3 완료: 워크플로우 구성 완료


---
# 테스트
---

## 시스템 실행 함수

In [14]:
def run_recommendation_system(user_query: str):
    initial_state = {
        "query": user_query,
        "search_results": "",
        "summary": "",
        "recommendations": "",
        "next_step": "search",
        "messages": []
    }

    print("=" * 70)
    print(f"사용자 질문: {user_query}")
    print("=" * 70)

    result = app.invoke(initial_state)

    print("\n" + "=" * 70)
    print("[1단계: 검색 결과]")
    print("=" * 70)
    print(result["search_results"])

    print("\n" + "=" * 70)
    print("[2단계: 논문 분석]")
    print("=" * 70)
    print(result["summary"])

    print("\n" + "=" * 70)
    print("[3단계: 최종 추천 Top 3]")
    print("=" * 70)
    print(result["recommendations"])

    return result

## 테스트 1: 컴퓨터 비전

In [15]:
result = run_recommendation_system(
    "I need papers about deep learning for computer vision in autonomous driving"
)

사용자 질문: I need papers about deep learning for computer vision in autonomous driving

[검색 에이전트] 'I need papers about deep learning for computer vision in autonomous driving' 검색 중...

[요약 에이전트] 'I need papers about deep learning for computer vision in autonomous driving' 요약 중...

[추천 에이전트] 논문 추천 진행 중...

[1단계: 검색 결과]
검색 결과 (3개 논문):

[1] 제목: Learning Common Representation from RGB and Depth Images
    연도: 2018
    요약: We propose a new deep learning architecture for the tasks of semantic
segmentation and depth prediction from RGB-D images. We revise the state of art
...
    유사도: 0.405

[2] 제목: HybridNet: Integrating Model-based and Data-driven Learning to Predict Evolution of Dynamical Systems
    연도: 2018
    요약: The robotic systems continuously interact with complex dynamical systems in
the physical world. Reliable predictions of spatiotemporal evolution of th...
    유사도: 0.399

[3] 제목: Artificial Intelligence Approaches
    연도: 2019
    요약: Artificial Intelligence (AI) has received trem

## 테스트 2: 자연어 처리

In [16]:
result = run_recommendation_system(
    "Natural language processing for chatbot development with transformers"
)

사용자 질문: Natural language processing for chatbot development with transformers

[검색 에이전트] 'Natural language processing for chatbot development with transformers' 검색 중...

[요약 에이전트] 'Natural language processing for chatbot development with transformers' 요약 중...

[추천 에이전트] 논문 추천 진행 중...

[1단계: 검색 결과]
검색 결과 (3개 논문):

[1] 제목: Behaviour Trees for Creating Conversational Explanation Experiences
    연도: 2022
    요약: This paper presented an XAI system specification and an interactive dialogue
model to facilitate the creation of Explanation Experiences (EE). Such
sp...
    유사도: 0.540

[2] 제목: Decision-support for the Masses by Enabling Conversations with Open Data
    연도: 2018
    요약: Open data refers to data that is freely available for reuse. Although there
has been rapid increase in availability of open data to public in the last...
    유사도: 0.525

[3] 제목: Introduction to the iDian
    연도: 2010
    요약: The iDian (previously named as the Operation Agent System) is a framework
designed to enabl

## 결론

3-에이전트 논문 추천 시스템을 완성했다.

### 학습한 내용:
1. LangChain 도구 만들기 (@tool)
2. 상태 기반 에이전트 구현
3. LangGraph 워크플로우 구성
4. 에이전트 간 데이터 전달

### 시스템 흐름:
```
사용자 질문
    ↓
검색 에이전트 (5개 논문 검색)
    ↓
요약 에이전트 (논문 분석)
    ↓
추천 에이전트 (Top 3 추천)
    ↓
최종 결과
```